In [3]:
import numpy
import scipy


The first step is to write down the Bloch equation in its matrice representation:

$$
\frac{\mathrm{d}}{\mathrm{d} t} \mathbf{M} = \mathbf{A} \cdot \mathbf{M}
$$

where the magnetization vector reads

$$
\mathbf{M} = 
    \begin{bmatrix}
        M_x \\
        M_y \\
        M_z  
    \end{bmatrix}
$$
and the transition matrix reads
$$
\mathbf{A} = 
    \begin{bmatrix}
        -R_2            & \Delta\omega_z    & -\gamma B_{1,y} \\
        -\Delta\omega_z & -R_2              &  \gamma B_{1,x} \\
         \gamma B_{1,y} & -\gamma B_{1,x}   & -R_1
    \end{bmatrix}
$$


In order to add the phenomenological thermalization to the lattice, we have two ways:

Either we can add an additional term $\mathbf{R}$ to the Bloch equation such that:

$$
\frac{\mathrm{d}}{\mathrm{d} t} \mathbf{M} = \mathbf{A} \cdot \mathbf{M} + \mathbf{R}
$$

Or we can extend the dimension of $\mathbf{M}$, including the magnetization at thermal equilibrium:


$$
\mathbf{M} = 
    \begin{bmatrix}
        M_x \\
        M_y \\
        M_z \\
        M_{eq}
    \end{bmatrix}
$$

Consequently, the transiton matrix also expands its dimension:

$$
\mathbf{A} = 
    \begin{bmatrix}
        -R_2            & \Delta\omega_z    & -\gamma B_{1,y} \\
        -\Delta\omega_z & -R_2              &  \gamma B_{1,x} \\
         \gamma B_{1,y} & -\gamma B_{1,x}   & -R_1
    \end{bmatrix}
$$